In [1]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import numpy as np

eps = np.finfo(float).eps
from numpy import log2 as log

df = pd.read_csv("tennis.csv")
print("\n Given Play Tennis Data Set:\n\n", df)


 Given Play Tennis Data Set:

      Outlook Temperature Humidity    Wind play
0      Sunny         Hot     High    Weak   No
1      Sunny         Hot     High  Strong   No
2   Overcast         Hot     High    Weak  Yes
3       Rain        Mild     High    Weak  Yes
4       Rain        Cool   Normal    Weak  Yes
5       Rain        Cool   Normal  Strong   No
6   Overcast        Cool   Normal  Strong  Yes
7      Sunny        Mild     High    Weak   No
8      Sunny        Cool   Normal    Weak  Yes
9       Rain        Mild   Normal    Weak  Yes
10     Sunny        Mild   Normal  Strong  Yes
11  Overcast        Mild     High  Strong  Yes
12  Overcast         Hot   Normal    Weak  Yes
13      Rain        Mild     High  Strong   No


In [33]:
def find_entropy(df):
    # df.keys(): attributs name of df
    # -1 play
    Class = df.keys()[
        -1
    ]  # To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    # last colum of data, of unique Vale ohne Wiederholung, this gives all 'Yes' and 'No'
    for value in values:
        fraction = df[Class].value_counts()[value] / len(df[Class])
        # count the value yes and no / Länge von Dataset, relative Häufigkeit
        entropy += -fraction * np.log2(fraction)
    return entropy


def find_entropy_attribute(df, attribute):
    # e.g attribute ist outlook
    Class = df.keys()[
        -1
    ]  # To make the code generic, changing target variable class name
    target_variables = df[Class].unique()  # This gives all 'Yes' and 'No'
    variables = df[
        attribute
    ].unique()  # This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:  # Sunny,Overcast, Rain
        entropy = 0
        for target_variable in target_variables:  #'Yes' and 'No'
            # df[attribute]:df[outlook] first colum of dataframe
            num = len(
                df[attribute][df[attribute] == variable][df[Class] == target_variable]
                # df[outlook]:where df[attribute] is equal variable, first sunny, then overcast then rain
                # and also df[play] == yes
                # only positive sunny
            )
            den = len(df[attribute][df[attribute] == variable])
            # outlook aunny total sunny
            fraction = num / (den + eps)
            entropy += -fraction * log(fraction + eps)
        fraction2 = den / len(df)
        entropy2 += -fraction2 * entropy
    return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []  # information gain
    for key in df.keys()[
        :-1
    ]:  # anything but the final one (outlook, Temperature, Humidity...)
        # Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df) - find_entropy_attribute(df, key))
        # find entropy of entire Dataframe - the entropy of all the possiable values of paticule attrubute
    return df.keys()[:-1][np.argmax(IG)]
    # alle the keys except the target attribut but with the maximum value og IG


def get_subtable(df, node, value):
    # potion of csv.file and need subtable
    # node: attribute, value of the attribute
    return df[df[node] == value].reset_index(drop=True)
    #


def buildTree(df, tree=None):
    Class = df.keys()[-1]
    # Class muss groß geschrieben ??
    # To make the code generic, changing target variable class name
    # Here we build our decision tree
    # Get attribute with maximum information gain
    node = find_winner(df)
    print('node:' + str(node))
    # root Node, return the highest I.G
    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(
        df[node]
    )  # Create an empty dictionary to create tree, value: Low,Med and High are values
    print('attValue:' + str(attValue))
    if tree is None:
        tree = {}
        tree[node] = {}
        print('tree:' + str(tree))
        print('tree[node]:' + str(tree[node]))
        # tree {outlook:{} } dictionary of dictionarys
        # below: fill the dictionary
    # We make loop to construct a tree by calling this function recursively.
    # In this we check if the subset is pure and stops if it is pure.
    for value in attValue:
        subtable = get_subtable(df, node, value)
        # subtable with sunny
        clValue, counts = np.unique(subtable[Class], return_counts=True)
        # among those rows, check the colom 'play' with values and entries
        # two values yes and no and the counts
        # two of len(counts) means we have yes and nos
        if len(counts) == 1:  # Checking purity of subset
            print('value:' + str(value))
            tree[node][value] = clValue[0]
            print('tree[node][value]:' + str(tree[node][value]))
            print('tree:' + str(tree))
            # clvale ist a list contains yes and no
        else:
            print('value:' + str(value))
            print('recursion')
            print('value:' + str(value))
            tree[node][value] = buildTree(subtable)  # Calling the function recursively
            
            # tree[outlook][sunny]: calling the function with the remaining entries
    return tree


def func(test, tree, default=None):
    attribute = next(iter(tree))
    print(attribute)
    if test[attribute] in tree[attribute].keys():
        print(tree[attribute].keys())
        print(test[attribute])
        result = tree[attribute][test[attribute]]
        if isinstance(result, dict):
            return func(test, result)
        else:
            return result
    else:
        return default

In [34]:
import pprint

tree = buildTree(df)
pprint.pprint(tree)

node:Outlook
attValue:['Overcast' 'Rain' 'Sunny']
tree:{'Outlook': {}}
tree[node]:{}
value:Overcast
tree[node][value]:Yes
tree:{'Outlook': {'Overcast': 'Yes'}}
value:Rain
recursion
value:Rain
node:Wind
attValue:['Strong' 'Weak']
tree:{'Wind': {}}
tree[node]:{}
value:Strong
tree[node][value]:No
tree:{'Wind': {'Strong': 'No'}}
value:Weak
tree[node][value]:Yes
tree:{'Wind': {'Strong': 'No', 'Weak': 'Yes'}}
value:Sunny
recursion
value:Sunny
node:Humidity
attValue:['High' 'Normal']
tree:{'Humidity': {}}
tree[node]:{}
value:High
tree[node][value]:No
tree:{'Humidity': {'High': 'No'}}
value:Normal
tree[node][value]:Yes
tree:{'Humidity': {'High': 'No', 'Normal': 'Yes'}}
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [6]:
df1 = pd.read_csv("house-votes-84-with-header.csv")
df1

,class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


<IPython.core.display.Javascript object>

In [ ]:
tree1 = buildTree(df1)
pprint.pprint(tree1)

In [ ]:
test={'Outlook':'Sunny','Temperature':'Hot','Humidity':'High','Wind':'Weak'}
ans = func(test, tree)
print(ans)
